# 4.4 Cleaning and transforming Dask DataFrames

## *Subjects covered*

* Selecting and filtering data
* Creating and dropping columns
* Finding and fixing columns with missing values
* Indexing and sorting DataFrames
* Combining DataFrames using join and union operations

## *Content*

- [Working with indexes and axes](#Working-with-indexes-and-axes)
    - [Selecting columns from a DataFrame](#Selecting-columns-from-a-DataFrame)
    - [Dropping columns from a DataFrame](#Dropping-columns-from-a-DataFrame)
    - [Renaming columns in a DataFrame](#Renaming-columns-in-a-DataFrame)
    - [Selecting rows from a DataFrame](#Selecting-rows-from-a-DataFrame)
- [Dealing with missing values](#Dealing-with-missing-values)
    - [Counting missing values in a DataFrame](#Counting-missing-values-in-a-DataFrame)
    - [Dropping columns with missing values](#Dropping-columns-with-missing-values)
    - [Imputing missing values](#Imputing-missing-values)
    - [Dropping rows with missing data](#Dropping-rows-with-missing-data)
    - [Imputing columns with multiple missing values](#Imputing-columns-with-multiple-missing-values)
- [Recoding data](#Recoding-data)
- [Elementwise operations](#Elementwise-operations)
- [Filtering and reindexing DataFrames](#Filtering-and-reindexing-DataFrames)
- [Joining and concatenating DataFrames](#Joining-and-concatenating-DataFrames)
- [Writing data to text files](#Writing-data-to-text-files)

## Working with indexes and axes

* Data cleaning is an important part of any data science project because anomalies and outliers in the data can negatively influence many statistical analyses
* Could lead us to make bad conclusions about the data and build machine learning models that don’t stand up over time
* Important that the data are cleaned up as much as possible before moving on to exploratory analysis

* Some of the Dask DataFrame operations look exactly the same as those of Pandas DataFrame
* Some operations differ due to the distributed nature of Dask

In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np

dtypes = {
 'Date First Observed': np.str,
 'Days Parking In Effect    ': np.str,
 'Double Parking Violation': np.str,
 'Feet From Curb': np.float32,
 'From Hours In Effect': np.str,
 'House Number': np.str,
 'Hydrant Violation': np.str,
 'Intersecting Street': np.str,
 'Issue Date': np.str,
 'Issuer Code': np.float32,
 'Issuer Command': np.str,
 'Issuer Precinct': np.float32,
 'Issuer Squad': np.str,
 'Issuing Agency': np.str,
 'Law Section': np.float32,
 'Meter Number': np.str,
 'No Standing or Stopping Violation': np.str,
 'Plate ID': np.str,
 'Plate Type': np.str,
 'Registration State': np.str,
 'Street Code1': np.uint32,
 'Street Code2': np.uint32,
 'Street Code3': np.uint32,
 'Street Name': np.str,
 'Sub Division': np.str,
 'Summons Number': np.uint32,
 'Time First Observed': np.str,
 'To Hours In Effect': np.str,
 'Unregistered Vehicle?': np.str,
 'Vehicle Body Type': np.str,
 'Vehicle Color': np.str,
 'Vehicle Expiration Date': np.str,
 'Vehicle Make': np.str,
 'Vehicle Year': np.float32,
 'Violation Code': np.uint16,
 'Violation County': np.str,
 'Violation Description': np.str,
 'Violation In Front Of Or Opposite': np.str,
 'Violation Legal Code': np.str,
 'Violation Location': np.str,
 'Violation Post Code': np.str,
 'Violation Precinct': np.float32,
 'Violation Time': np.str
}

nyc_data_raw = dd.read_csv('nyc-parking-tickets/*.csv', dtype=dtypes, usecols=dtypes.keys())

* Code above should look familiar from previous notebook
* Using an explicit schema `dtypes` the data from four CSV files is loaded into DataFrame `nyc_data_raw`
* Selecting the wanted common columns using `usecols=dtypes.keys()`

**Explore dataset by using indices and axes in DataFrame**

<img src="./images/DataFrame_structure.png" width="700"/>

### Selecting columns from a DataFrame

In [2]:
with ProgressBar():
    display(nyc_data_raw['Plate ID'].head())

[########################################] | 100% Completed |  2.0s


0    GBB9093
1    62416MB
2    78755JZ
3    63009MA
4    91648MC
Name: Plate ID, dtype: object

* Selecting one column returns a Dask Series
* Indices are copied over from Dask DataFrame

In [3]:
with ProgressBar():
    display(nyc_data_raw[['Plate ID', 'Registration State']].head())

[########################################] | 100% Completed |  2.1s


,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


In [4]:
columns_to_select = ['Plate ID', 'Registration State']

with ProgressBar():
    display(nyc_data_raw[columns_to_select].head())

[########################################] | 100% Completed |  2.2s


,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


### Dropping columns from a DataFrame

In [5]:
# Listing 5.5
with ProgressBar():
    display(nyc_data_raw.drop('Violation Code', axis=1).head())

[########################################] | 100% Completed |  2.2s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1283294138,GBB9093,NY,PAS,08/04/2013,SUBN,AUDI,P,37250,13610,...,GY,0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN
1,1283294151,62416MB,NY,COM,08/04/2013,VAN,FORD,P,37290,40404,...,WH,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN
2,1283294163,78755JZ,NY,COM,08/05/2013,P-U,CHEVR,P,37030,31190,...,NaN,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
3,1283294175,63009MA,NY,COM,08/05/2013,VAN,FORD,P,37270,11710,...,WH,0,2010.0,-,0.0,NaN,NaN,NaN,NaN,NaN
4,1283294187,91648MC,NY,COM,08/08/2013,TRLR,GMC,P,37240,12010,...,BR,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN


* As with Pandas, Dask defaults to axis 0
* If user forgets to use `axis=1`, Dask will try to locate and drop a row with an index of `Violation Code`
* However, dropping rows is not implemented in Dask and will return error `NotImplementedError: Drop currently only works for axis=1`

**Dropping multiple columns from a DataFrame**

In [6]:
violationColumnNames = list(filter(lambda columnName: 'Violation' in columnName, nyc_data_raw.columns))

with ProgressBar():
    display(nyc_data_raw.drop(violationColumnNames, axis=1).head())

[########################################] | 100% Completed |  2.1s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb
0,1283294138,GBB9093,NY,PAS,08/04/2013,SUBN,AUDI,P,37250,13610,...,408.0,F1,BBBBBBB,ALL,ALL,GY,0,2013.0,-,0.0
1,1283294151,62416MB,NY,COM,08/04/2013,VAN,FORD,P,37290,40404,...,408.0,C,BBBBBBB,ALL,ALL,WH,0,2012.0,-,0.0
2,1283294163,78755JZ,NY,COM,08/05/2013,P-U,CHEVR,P,37030,31190,...,408.0,F7,BBBBBBB,ALL,ALL,NaN,0,0.0,-,0.0
3,1283294175,63009MA,NY,COM,08/05/2013,VAN,FORD,P,37270,11710,...,408.0,F1,BBBBBBB,ALL,ALL,WH,0,2010.0,-,0.0
4,1283294187,91648MC,NY,COM,08/08/2013,TRLR,GMC,P,37240,12010,...,408.0,E1,BBBBBBB,ALL,ALL,BR,0,2012.0,-,0.0


### Renaming columns in a DataFrame

In [7]:
nyc_data_renamed = nyc_data_raw.rename(columns={'Plate ID':'License Plate'})
nyc_data_renamed

,Summons Number,License Plate,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
npartitions=142,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,object,object,object,object,uint16,object,object,object,uint32,uint32,uint32,object,object,float32,float32,float32,object,object,object,object,object,object,object,object,object,object,float32,object,object,object,object,object,object,object,float32,object,float32,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Selecting rows froma DataFrame

* Common situations where you know the range of rows you want to retrieve
* For example when data is indexed by a date or time
* Selecting rows by index is the appropriate way to get the data
* Selecting data by index offers a performance boost over search and filter methods

In [8]:
with ProgressBar():
    display(nyc_data_raw.loc[56].head(1))

[########################################] | 100% Completed |  2.1s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
56,1293090530,GES3519,NY,PAS,07/07/2013,40,SDN,HONDA,F,70630,...,BLACK,0,1997.0,-,0.0,NaN,NaN,NaN,NaN,NaN


* Use a slice to return rows between index 100 and 200
* However, not possible to select every other row in that range, since `drop` for dropping rows is not implemented in Dask DataFrame

In [9]:
with ProgressBar():
    display(nyc_data_raw.loc[100:200].head(100))

[########################################] | 100% Completed |  2.1s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
100,1294727205,XBD7628,VA,PAS,08/04/2013,17,SUBN,JEEP,P,14510,...,GRY,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
101,1294727461,R613159,IL,PAS,07/17/2013,17,SDN,VOLKS,P,14510,...,BLUE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
102,1294727473,6TCX735,CA,PAS,07/18/2013,17,SDN,MAZDA,P,14510,...,BLACK,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
103,1294727497,ZWZ43K,NJ,PAS,08/10/2013,17,SUBN,LINCO,P,14510,...,NaN,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
104,1295357240,T624858C,NY,PAS,07/22/2013,21,SUBN,TOYOT,X,28790,...,SILVE,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN
105,1295546486,42909JM,NY,COM,07/17/2013,19,DELV,MERCU,P,58590,...,BROWN,0,1996.0,-,0.0,NaN,NaN,NaN,NaN,NaN
106,1302446411,EPS8277,NY,PAS,06/18/2013,17,SDN,BMW,C,20110,...,BLACK,0,2005.0,-,0.0,NaN,NaN,NaN,NaN,NaN
107,1302446484,452WW4,MA,PAS,06/24/2013,46,SDN,CHEVR,C,75530,...,YELLO,0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN
108,1302446538,4274WY,CT,PAS,07/10/2013,46,SDN,NISSA,C,75530,...,GOLD,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
109,1302453774,GEX5296,NY,PAS,07/26/2013,14,SDN,ME/BE,C,36420,...,BLACK,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN


* Workaround: to retrieve every other row in the extracted range, use filtering in Pandas DataFrame
* In code below, `some_rows` is actually a Pandas DataFrame
* This is because `.head()` triggers a computation and therefore returns a Pandas DataFrame
* ==> Now possible to use `.drop(range(100, 200, 2))`

**Warning**

* Be aware that extracted range may be too large for a Pandas DataFrame and cause a out-of-memory error
* This process is suitable only for fairly small slice of Dask DataFrame

In [10]:
with ProgressBar():
    some_rows = nyc_data_raw.loc[100:200].head(100)
some_rows.drop(range(100, 200, 2))

[########################################] | 100% Completed |  2.1s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
101,1294727461,R613159,IL,PAS,07/17/2013,17,SDN,VOLKS,P,14510,...,BLUE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
103,1294727497,ZWZ43K,NJ,PAS,08/10/2013,17,SUBN,LINCO,P,14510,...,NaN,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
105,1295546486,42909JM,NY,COM,07/17/2013,19,DELV,MERCU,P,58590,...,BROWN,0,1996.0,-,0.0,NaN,NaN,NaN,NaN,NaN
107,1302446484,452WW4,MA,PAS,06/24/2013,46,SDN,CHEVR,C,75530,...,YELLO,0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN
109,1302453774,GEX5296,NY,PAS,07/26/2013,14,SDN,ME/BE,C,36420,...,BLACK,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
111,1302453798,867ZFX,CT,PAS,07/15/2013,17,SDN,ACURA,C,0,...,WHITE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
113,1302456593,GEZ1408,NY,PAS,07/15/2013,17,SUBN,HYUND,X,77730,...,BLK,0,2013.0,-,0.0,NaN,NaN,NaN,NaN,NaN
115,1302456738,UCSLIM,NY,PAS,08/07/2013,17,SUBN,FORD,X,77730,...,BLK,0,2003.0,-,0.0,NaN,NaN,NaN,NaN,NaN
117,1302466203,HPM2636,PA,PAS,08/08/2013,40,SUBN,CADIL,X,24245,...,SILVR,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
119,1302466227,TWODAMAX,NY,SRF,08/05/2013,17,SDN,HYUND,X,53630,...,GRAY,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN


## Dealing with missing values

Part of the data cleaning process is to take care of missing values

**Reasons for missing data**

* Deficiencies in the data collection process
* Evolving needs over time
* Data processing issues
* Storage issues

**Data-quality issues remedy**

* Remove the rows/columns with missing data from your dataset
* Assign the missing values a default value
* Impute the missing values

**Which remedy to choose?**

Depends on 
* the objective of your analysis
* the context
* the domain
* the structure of missingness


**Good rule of thumb**

* Work with the stakeholders who will be interpreting and using your analyses
* Agree upon what makes the most sense in the context of the problem you’re trying to solve

Examples below show how to move forward with any of the three remedies mentioned above.

### Counting missing values in a DataFrame

**Calculating the percentage of missing values by column**

In [11]:
missing_values = nyc_data_raw.isnull().sum()
with ProgressBar():
    percent_missing = ((missing_values / nyc_data_raw.index.size) * 100).compute()
percent_missing

[########################################] | 100% Completed |  4min 42.6s


Summons Number                        0.000000
Plate ID                              0.020867
Registration State                    0.000000
Plate Type                            0.000000
Issue Date                            0.000000
Violation Code                        0.000000
Vehicle Body Type                     0.564922
Vehicle Make                          0.650526
Issuing Agency                        0.000000
Street Code1                          0.000000
Street Code2                          0.000000
Street Code3                          0.000000
Vehicle Expiration Date               0.000002
Violation Location                   15.142846
Violation Precinct                    0.000002
Issuer Precinct                       0.000002
Issuer Code                           0.000002
Issuer Command                       15.018851
Issuer Squad                         15.022566
Violation Time                        0.019207
Time First Observed                  90.040886
Violation Cou

### Dropping columns with missing values

**Dropping columns that have more than 50% missing values**

In [12]:
columns_to_drop = list(percent_missing[percent_missing >= 50].index)
nyc_data_clean_stage1 = nyc_data_raw.drop(columns_to_drop, axis=1)

* Again, dropping columns with 50% or more missing values is an arbitrary level.
* Need to be adjusted for the specific domain 

### Imputing missing values

**Imputing**

Use the data that is available to make a reasonable guess at what the missing data might be.

In example below, use most frequent value in a column to replace `np.nan` in column `Vehicle Color` with 1.152299% missing values

In [13]:
with ProgressBar():
    count_of_vehicle_colors = nyc_data_clean_stage1['Vehicle Color'].value_counts().compute()
most_common_color = count_of_vehicle_colors.sort_values(ascending=False).index[0]

# Fill missing vehicle color with the most common color
nyc_data_clean_stage2 = nyc_data_clean_stage1.fillna({'Vehicle Color': most_common_color})

[########################################] | 100% Completed |  4min 13.8s


In [31]:
count_of_vehicle_colors

GY       6280314
WH       6074770
WHITE    5624960
BK       5121030
BLACK    2758479
BL       2193035
GREY     1668739
RD       1383881
SILVE    1253287
BROWN    1194809
BLUE     1178034
RED      1060329
GR        875167
TN        575236
GREEN     561270
OTHER     434107
YW        343838
BR        341887
BLK       318174
GRAY      278593
TAN       242193
GL        238333
GOLD      191405
YELLO     190671
GRY       147129
MR        140622
WHT       132713
WT         87742
ORANG      63572
OR         54614
          ...   
MARUE          1
MARUI          1
MBWC           1
METBL          1
METBK          1
MET/O          1
MERWH          1
MERON          1
MERL           1
MERG           1
MEDS           1
MDE            1
MD-BL          1
MCNY           1
MCCT           1
MBROW          1
MARVN          1
MBR            1
MAZOO          1
MAZON          1
MAXOO          1
MAX            1
MAWE           1
MAVEN          1
MAUL           1
MAU            1
MATOO          1
MATH          

* This means that all missing colours are replaced by the most frequent colour (GY) using the `.fillna` method

### Dropping rows with missing data

* Now identify rows with missing values in columns that have less than 5% missing values
* Remove those rows from the data
* Now can use these columns for computations

In [14]:
rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
nyc_data_clean_stage3 = nyc_data_clean_stage2.dropna(subset=rows_to_drop)

Note that no rows will be dropped on account of missing vehicle colors because they’ve already been filled in.

### Imputing multiple columns with missing values

* Now check what columns are left to be cleaned up and what their datatypes are
* For any columns that have more than 5% missing values and less than 50% missing values, fill the missing values with a default value

**Finding the datatypes of the remaining columns**

In [15]:
remaining_columns_to_clean = list(percent_missing[(percent_missing >= 5) & (percent_missing < 50)].index)
nyc_data_raw.dtypes[remaining_columns_to_clean]

Violation Location                   object
Issuer Command                       object
Issuer Squad                         object
Violation County                     object
Violation In Front Of Or Opposite    object
House Number                         object
Days Parking In Effect               object
From Hours In Effect                 object
To Hours In Effect                   object
Violation Post Code                  object
Violation Description                object
dtype: object

* Note that even though these columns were defined as `np.str`in the schema they now show as `object`
* No need to worry, this is just cosmetic
* Dask only explicitly shows numeric (`int`, `float`, and so on) datatypes
* Any non-numeric datatypes will show as `object`

**Making a dictionary of values for `fillna`**

In [16]:
unknown_default_dict = dict(map(lambda columnName: (columnName, 'Unknown'), remaining_columns_to_clean))

**Filling the DataFrame with default values**

In [17]:
nyc_data_clean_stage4 = nyc_data_clean_stage3.fillna(unknown_default_dict)

In [18]:
with ProgressBar():
    print(nyc_data_clean_stage4.isnull().sum().compute())
    nyc_data_clean_stage4.persist()

[########################################] | 100% Completed |  6min 20.0s
Summons Number                       0
Plate ID                             0
Registration State                   0
Plate Type                           0
Issue Date                           0
Violation Code                       0
Vehicle Body Type                    0
Vehicle Make                         0
Issuing Agency                       0
Street Code1                         0
Street Code2                         0
Street Code3                         0
Vehicle Expiration Date              0
Violation Location                   0
Violation Precinct                   0
Issuer Precinct                      0
Issuer Code                          0
Issuer Command                       0
Issuer Squad                         0
Violation Time                       0
Violation County                     0
Violation In Front Of Or Opposite    0
House Number                         0
Street Name                  

* In the remaining columns there are no more missing data
* Now it would be an opportune time to persist DataFrame `nyc_data_clean_stage4`

## Recoding data

* There may be some instances in the data whose validity is suspect
* Need to have a way to clean up those kinds of data anomalies
* Option 1 - recoding those values to a more likely choice (such as the most frequent value or arithmetic mean)
* Option 2 - placing the anomalous data in an `Other` category

In [19]:
with ProgressBar():
    license_plate_types = nyc_data_clean_stage4['Plate Type'].value_counts().compute()
license_plate_types

[########################################] | 100% Completed |  5min 20.3s


PAS    30452502
COM     7966914
OMT     1389341
SRF      394656
OMS      368952
IRP      234688
999      209778
TRC      106423
MOT      103909
OMR       75148
ORG       64960
CMB       56175
APP       54000
OML       52406
MED       50515
SPO       32402
PSD       28627
TOW       22862
SCL       22726
RGL       20721
DLR       16691
VAS       15563
SRN       14221
TRA       13763
ITP       12111
CMH        8513
TRL        7941
CSP        5402
MCL        5105
PHS        3569
         ...   
SUP         130
VPL         129
ATD         128
OMO         120
NYS         108
FPW         106
CCK         105
LUA          71
FAR          70
LMC          59
HIR          51
PPH          49
CLG          43
ATV          40
GAC          32
GSM          30
JSC          27
HAC          26
HSM          25
JCA          22
BOT          20
WUG          13
JCL          12
USS          11
GSC           7
HOU           4
JWV           3
LOC           3
HIF           2
SNO           2
Name: Plate Type, Length

`* The vast majority of license plate types are `PAS (passenger vehicles) `COM` (commercial vehicles)
* These two plate types make up over 92% of the entire DataFrame (~38M out of ~41M rows)
* Can also see that there are 90 distinct license plate types (Length: 90)
* Try to collapse the `Plate Type` column such that there are only three types: `PAS`, `COM`, and `Other`

In [20]:
condition = nyc_data_clean_stage4['Plate Type'].isin(['PAS', 'COM'])
plate_type_masked = nyc_data_clean_stage4['Plate Type'].where(condition, 'Other')
nyc_data_recode_stage1 = nyc_data_clean_stage4.drop('Plate Type', axis=1)
nyc_data_recode_stage2 = nyc_data_recode_stage1.assign(PlateType=plate_type_masked)
nyc_data_recode_stage3 = nyc_data_recode_stage2.rename(columns={'PlateType':'Plate Type'})

In [21]:
with ProgressBar():
    display(nyc_data_recode_stage3['Plate Type'].value_counts().compute())

[########################################] | 100% Completed |  6min 11.8s


PAS      30452502
COM       7966914
Other     3418586
Name: Plate Type, dtype: int64

In [22]:
single_color = list(count_of_vehicle_colors[count_of_vehicle_colors == 1].index)
condition = nyc_data_clean_stage4['Vehicle Color'].isin(single_color)
vehicle_color_masked = nyc_data_clean_stage4['Vehicle Color'].mask(condition, 'Other')
nyc_data_recode_stage4 = nyc_data_recode_stage3.drop('Vehicle Color', axis=1)
nyc_data_recode_stage5 = nyc_data_recode_stage4.assign(VehicleColor=vehicle_color_masked)
nyc_data_recode_stage6 = nyc_data_recode_stage5.rename(columns={'VehicleColor':'Vehicle Color'})

## Elementwise operations

In [23]:
from datetime import datetime
issue_date_parsed = nyc_data_recode_stage6['Issue Date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"), meta=datetime)
nyc_data_derived_stage1 = nyc_data_recode_stage6.drop('Issue Date', axis=1)
nyc_data_derived_stage2 = nyc_data_derived_stage1.assign(IssueDate=issue_date_parsed)
nyc_data_derived_stage3 = nyc_data_derived_stage2.rename(columns={'IssueDate':'Issue Date'})

In [24]:
with ProgressBar():
    display(nyc_data_derived_stage3['Issue Date'].head())

[########################################] | 100% Completed |  6.7s


0   2013-08-04
1   2013-08-04
2   2013-08-05
3   2013-08-05
4   2013-08-08
Name: Issue Date, dtype: datetime64[ns]

In [25]:
issue_date_month_year = nyc_data_derived_stage3['Issue Date'].apply(lambda dt: dt.strftime("%Y%m"), meta=int)
nyc_data_derived_stage4 = nyc_data_derived_stage3.assign(IssueMonthYear=issue_date_month_year)
nyc_data_derived_stage5 = nyc_data_derived_stage4.rename(columns={'IssueMonthYear':'Citation Issued Month Year'})

In [26]:
with ProgressBar():
    display(nyc_data_derived_stage5['Citation Issued Month Year'].head())

[########################################] | 100% Completed |  8.9s


0    201308
1    201308
2    201308
3    201308
4    201308
Name: Citation Issued Month Year, dtype: object

## Filtering and reindexing DataFrames

In [27]:
months = ['201310','201410','201510','201610','201710']
condition = nyc_data_derived_stage5['Citation Issued Month Year'].isin(months)
october_citations = nyc_data_derived_stage5[condition]

with ProgressBar():
    display(october_citations.head())

[########################################] | 100% Completed |  8.9s


,Summons Number,Plate ID,Registration State,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,...,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Issue Date,Citation Issued Month Year
1609,1340313923,GEK8055,NY,40,SUBN,HONDA,P,79630,40404,40404,...,ALL,ALL,2013.0,5.0,Unknown,Unknown,PAS,BROWN,2013-10-23,201310
23367,1351679867,XE726658,DE,20,P-U,DODGE,P,90980,0,0,...,ALL,ALL,2002.0,0.0,Unknown,Unknown,PAS,RED,2013-10-21,201310
24172,1351805253,42067JM,NY,14,DELV,FRUEH,P,25630,13610,24985,...,ALL,ALL,1999.0,0.0,Unknown,Unknown,COM,WHITE,2013-10-17,201310
32902,1355051060,76254JY,NY,46,DELV,FRUEH,P,68020,26760,66120,...,ALL,ALL,2007.0,0.0,Unknown,Unknown,COM,WHITE,2013-10-10,201310
32903,1355051071,44125MC,NY,46,VAN,FORD,P,68020,26490,26520,...,ALL,ALL,2011.0,0.0,Unknown,Unknown,COM,WHITE,2013-10-10,201310


In [28]:
bound_date = '2016-4-25'
condition = nyc_data_derived_stage5['Issue Date'] > bound_date
citations_after_bound = nyc_data_derived_stage5[condition]

with ProgressBar():
    display(citations_after_bound.head())

[########################################] | 100% Completed |  9.1s


,Summons Number,Plate ID,Registration State,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,...,From Hours In Effect,To Hours In Effect,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Plate Type,Vehicle Color,Issue Date,Citation Issued Month Year
3741,1346495701,GCJ8613,NY,20,SDN,DODGE,X,10210,19210,19250,...,ALL,ALL,2010.0,0.0,Unknown,Unknown,PAS,BLK,2017-08-04,201708
3748,1346551819,GHJ2373,NY,20,SUBN,FORD,X,0,0,0,...,ALL,ALL,2010.0,0.0,Unknown,Unknown,PAS,BLK,2019-07-22,201907
6905,1348077426,66811MB,NY,78,IR,FRUEH,P,25680,46320,8120,...,0900P,0500A,2007.0,0.0,Unknown,Unknown,COM,WHT,2053-08-02,205308
12967,1353305650,91609MC,NY,46,VAN,FORD,T,24890,18670,18690,...,ALL,ALL,2012.0,0.0,Unknown,Unknown,COM,BR,2016-08-07,201608
17144,1354617988,54015JV,NY,78,VAN,FORD,P,38590,50150,52290,...,0900P,0500A,2007.0,0.0,Unknown,Unknown,COM,WHITE,2032-07-28,203207


In [29]:
with ProgressBar():
    condition = (nyc_data_derived_stage5['Issue Date'] > '2014-01-01') & (nyc_data_derived_stage5['Issue Date'] <= '2017-12-31')
    nyc_data_filtered = nyc_data_derived_stage5[condition]
    nyc_data_new_index = nyc_data_filtered.set_index('Citation Issued Month Year')

[########################################] | 100% Completed | 32min 46.0s


## Joining and concatenating DataFrames

### Joining two DataFrames

In [ ]:
import pandas as pd
nyc_temps = pd.read_csv('nyc-temp-data.csv')
nyc_temps_indexed = nyc_temps.set_index(nyc_temps.monthYear.astype(str))

nyc_data_with_temps = nyc_data_new_index.join(nyc_temps_indexed, how='inner')

with ProgressBar():
    display(nyc_data_with_temps.head(15))

### Unioning two DataFrames

In [ ]:
fy16 = dd.read_csv('nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv', dtype=dtypes, usecols=dtypes.keys())
fy17 = dd.read_csv('nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2017.csv', dtype=dtypes, usecols=dtypes.keys())

fy1617 = fy16.append(fy17)

with ProgressBar():
    print(fy16['Summons Number'].count().compute())

with ProgressBar():
    print(fy17['Summons Number'].count().compute())

with ProgressBar():
    print(fy1617['Summons Number'].count().compute())


## Writing data to text files

In [ ]:
# Listing 5.33
with ProgressBar():
    if not os.path.exists('nyc-final-csv'):
        os.makedirs('nyc-final-csv') 
    nyc_data_with_temps.repartition(npartitions=1).to_csv('nyc-final-csv/part*.csv')

In [ ]:
# Listing 5.33
with ProgressBar():
    if not os.path.exists('nyc-final-csv-compressed'):
        os.makedirs('nyc-final-csv-compressed')
    nyc_data_with_temps.to_csv(
        filename='nyc-final-csv-compressed/*', 
        compression='gzip', 
        sep='|', 
        na_rep='NULL', 
        header=False, 
        index=False)